In [10]:
import pandas as pd

In [11]:
metadata = pd.read_csv("/FastData/czirion/Crypto_Diversity_Pipeline/results_joined_241204/02.Dataset/metadata.csv", header=0)

In [12]:
runs = pd.read_csv("/BigData/czirion/Crypto_Diversity_Pipeline/data/runs_table.tsv", sep="\t", header=0)

In [13]:
missing_runs = pd.DataFrame({
    'run': ["SRR798282", "SRR797784", "SRR797768", "SRR798708", "SRR798752", "SRR798587", "SRR798870", "SRR836666", "SRR642859", "SRR836892", "SRR1063256", "SRR1942909"],
    'strain': ["Bt161", "Gb159-1", "Bt2", "Bt209", "Bt59", "A3-38-20", "MW-RSA606", "AD5-53a", "AD1-83a", "In2632", "LP-RSA2297", "PMHc1032.ENR"]
})

Get the runs present in the Ashton dataset

In [14]:
metadata_ashton = pd.read_csv("/FastData/czirion/Crypto_Diversity_Pipeline/Crypto_Ashton/config/paper_metadata.csv", header=0)
metadata_ashton.rename(columns={"Sequencing ID":"run", "Sub-clade":"vni_subclade"}, inplace=True)
metadata_ashton = metadata_ashton[["run", "vni_subclade"]]
metadata_ashton = metadata_ashton[metadata_ashton['run'].isin(runs['run'])| metadata_ashton['run'].isin(missing_runs['run'])]
metadata_ashton

,run,vni_subclade
699,SRR1063253,VNIb
700,SRR1063256,VNIa-4
701,SRR1063265,VNIc
702,SRR1063269,VNIc
703,SRR1063286,VNIb
...,...,...
879,SRR836988,VNIc
880,SRR837084,VNIb
881,SRR837099,VNIa-4
882,SRR842038,VNIb


In [15]:
runs = runs[runs['run'].isin(metadata_ashton['run'])]
runs

,sample,run
0,SRS403287,SRR642858
6,SRS404442,SRR799061
8,SRS404443,SRR798290
15,SRS404444,SRR797707
19,SRS404447,SRR797736
...,...,...
1599,SRS885874,SRR1942967
1608,SRS885879,SRR1942971
1617,SRS885884,SRR1942988
1625,SRS885889,SRR1943005


Add the runs to the complete metadata

In [16]:
metadata_complete = metadata.copy()
metadata_complete = metadata_complete.merge(runs[['sample', 'run']], on='sample', how='left')
metadata_complete['run'] = metadata_complete.apply(
    lambda row: missing_runs.loc[missing_runs['strain'] == row['strain'], 'run'].values[0] 
    if row['strain'] in missing_runs['strain'].values else row['run'], axis=1)

Add the subclade to the complete metadata

In [17]:
metadata_complete = metadata_complete.merge(metadata_ashton, on='run', how='left')

Add the Ashton runs to the complete metadata

In [18]:
metadata_complete['run'] = metadata_complete.apply(
    lambda row: row['sra_accession'] if row['dataset'] == 'Ashton' and pd.notna(row['sra_accession']) else row['run'], axis=1)
metadata_complete

,strain,sample,name,lineage,vni_subdivision,mating_type,country_of_origin,isolation_source,broad_project,sra_accession,...,dataset,species_id_from_mash_anlaysis,study,hiv_status,continent_of_origin,year_of_origin,mean_depth_of_mapping_with_mq_>_30_across_whole_genome,proportion_of_genome_covered_by_at_least_5_reads_which_mapped_with_mq_>_30,run,vni_subclade
0,NRHc5005.ENR,SRS881178,NRHc5005.ENR,VNBI,NaN,α,Botswana,Clinical,G84316,"SRR1925990, SRR1926071, SRR1944719",...,Desjardins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ftc202-1,SRS881184,Ftc202-1,VNBI,NaN,α,Botswana,Mopane tree,G84227,"SRR1925933, SRR1926009, SRR1942977",...,Desjardins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ftc239-1,SRS881232,Ftc239-1,VNBI,NaN,α,Botswana,Mopane tree,G84238,"SRR1926034, SRR1931691, SRR1942979, SRR1944785",...,Desjardins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ftc137-1,SRS885855,Ftc137-1,VNBI,NaN,α,Botswana,Mopane tree,G84214,"SRR1942921, SRR1942926, SRR1944803",...,Desjardins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NRHc5045.ENR.CLIN.ISO,SRS885893,NRHc5045.ENR.CLIN.ISO,VNBI,NaN,a,Botswana,Clinical,G84357,"SRR1943021, SRR1947212, SRR1947255",...,Desjardins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,UI_16771,ERS2541344,UI_16771,VNI,VNIa-4,NaN,Laos,NaN,NaN,ERR2624157,...,Ashton,Cryptococcus neoformans var. grubii H99,Ashton,negative,Asia,2010.0,53.377328,0.985342,ERR2624157,NaN
1066,UI_21616,ERS2541343,UI_21616,VNI,VNIa-4,NaN,Laos,NaN,NaN,ERR2624274,...,Ashton,Cryptococcus neoformans var. grubii H99,Ashton,positive,Asia,2012.0,49.318056,0.985419,ERR2624274,NaN
1067,UI_31647-2,ERS2541342,UI_31647-2,VNI,VNIa-4,NaN,Laos,NaN,NaN,ERR2624321,...,Ashton,Cryptococcus neoformans var. grubii H99,Ashton,positive,Asia,NaN,44.757365,0.986385,ERR2624321,NaN
1068,UI29674,ERS2541318,UI29674,VNI,VNIa-4,NaN,Laos,NaN,NaN,ERR2624436,...,Ashton,Cryptococcus neoformans var. grubii H99,Ashton,positive,Asia,NaN,54.533178,0.985632,ERR2624436,NaN


In [23]:
subclade = metadata_complete[["sample","strain","name","dataset","run", "source","lineage", "vni_subdivision", "vni_subclade"]]

In [24]:
subclade['vni_subdivision'] = subclade.apply(
    lambda row: row['vni_subclade'] if row['dataset'] == 'Desjardins' and pd.notnull(row['vni_subclade']) else row['vni_subdivision'], axis=1)
subclade

/tmp/ipykernel_3674243/3024724193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subclade['vni_subdivision'] = subclade.apply(


,sample,strain,name,dataset,run,source,lineage,vni_subdivision,vni_subclade
0,SRS881178,NRHc5005.ENR,NRHc5005.ENR,Desjardins,NaN,Clinical,VNBI,NaN,NaN
1,SRS881184,Ftc202-1,Ftc202-1,Desjardins,NaN,Environment,VNBI,NaN,NaN
2,SRS881232,Ftc239-1,Ftc239-1,Desjardins,NaN,Environment,VNBI,NaN,NaN
3,SRS885855,Ftc137-1,Ftc137-1,Desjardins,NaN,Environment,VNBI,NaN,NaN
4,SRS885893,NRHc5045.ENR.CLIN.ISO,NRHc5045.ENR.CLIN.ISO,Desjardins,NaN,Clinical,VNBI,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1065,ERS2541344,UI_16771,UI_16771,Ashton,ERR2624157,Clinical,VNI,VNIa-4,NaN
1066,ERS2541343,UI_21616,UI_21616,Ashton,ERR2624274,Clinical,VNI,VNIa-4,NaN
1067,ERS2541342,UI_31647-2,UI_31647-2,Ashton,ERR2624321,Clinical,VNI,VNIa-4,NaN
1068,ERS2541318,UI29674,UI29674,Ashton,ERR2624436,Clinical,VNI,VNIa-4,NaN


In [25]:
new_row = pd.DataFrame([{'sample': 'GCF_000149245', 'strain': 'H99', 'name': 'H99', 'dataset': '', 'run': 'GCF_000149245', 'source': 'Clinical', 'lineage': 'VNI', 'vni_subdivision': 'VNIb', 'vni_subclade': 'VNIb'}])
subclade = pd.concat([subclade, new_row], ignore_index=True)
subclade = subclade.drop(columns=['vni_subclade'])

In [27]:
subclade.to_csv("/FastData/czirion/Crypto_Diversity_Pipeline/analyses/cnv_plot/desjardins/data/metadata_fixed.csv", index=False)
subclade

,sample,strain,name,dataset,run,source,lineage,vni_subdivision
0,SRS881178,NRHc5005.ENR,NRHc5005.ENR,Desjardins,NaN,Clinical,VNBI,NaN
1,SRS881184,Ftc202-1,Ftc202-1,Desjardins,NaN,Environment,VNBI,NaN
2,SRS881232,Ftc239-1,Ftc239-1,Desjardins,NaN,Environment,VNBI,NaN
3,SRS885855,Ftc137-1,Ftc137-1,Desjardins,NaN,Environment,VNBI,NaN
4,SRS885893,NRHc5045.ENR.CLIN.ISO,NRHc5045.ENR.CLIN.ISO,Desjardins,NaN,Clinical,VNBI,NaN
...,...,...,...,...,...,...,...,...
1066,ERS2541343,UI_21616,UI_21616,Ashton,ERR2624274,Clinical,VNI,VNIa-4
1067,ERS2541342,UI_31647-2,UI_31647-2,Ashton,ERR2624321,Clinical,VNI,VNIa-4
1068,ERS2541318,UI29674,UI29674,Ashton,ERR2624436,Clinical,VNI,VNIa-4
1069,ERS2541323,UI32312,UI32312,Ashton,ERR2624374,Clinical,VNI,VNIa-4
